### Importing Libraries

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
import librosa
import librosa.display
import IPython.display
import os
import cv2
from os import listdir

from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, BatchNormalization
from keras import backend as K
from keras.models import load_model
from keras.models import Model
from keras.utils import img_to_array, load_img
from keras.utils import to_categorical
import numpy as np
import tensorflow as tf


from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc 

import matplotlib
import gc
matplotlib.use('Agg') # No pictures displayed 
import pandas as pd

In [ ]:
#### Initializing the LabelEncoder from sklearn.preprocessing to convert the labels into categorical encoding

In [2]:
labelencoder = LabelEncoder()

#### Defining the Convolutional Neural Network Model
Here, the input shape is set to be (128,128) and 3 color channels. Further, the samples are loaded with a Batch size of 32, the Learning rate is set to 1e-3. Also, the model is compiled with the 'adam' optimizer and binary_crossentropy as the classification loss.

The model consists of 3 Convolution layers followed by 3 Fully Connected dense layers as classifier. The Convolution layers have activation function as 'Relu' (Rectified Linear Unit). Morevoer the last dense layer possesses 'sigmoid' as activation function. Each Convolution layer consists of a Maxpooling2D and BatchNormalization followed by a Dropout layer(p = 0.25)

In [3]:
def create_model():
    num_rows = 128
    num_columns = 128
    num_channels = 3
    inputShape = (num_rows, num_columns, num_channels)
    chanDim = -1
    LR = 1e-3
    BATCH_SIZE = 32
    if K.image_data_format() == "channels_first":
        inputShape = (num_rows, num_columns, num_channels)
        chanDim = 1

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=inputShape))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics=["accuracy"])
    
    return model


In [4]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix

def compute_confusion_matrix(y_true, y_pred,normalize=False):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    return cm

In [7]:
path = r"C:\Users\rudra\OneDrive\Desktop\animals\OvR3"
animals = listdir(path)
print(animals)
print("number of classes:",len(animals))

['antelope', 'badger', 'bat', 'bear', 'bee', 'beetle', 'bison', 'boar', 'butterfly', 'cat', 'caterpillar', 'chimpanzee', 'cockroach', 'cow', 'coyote', 'crab', 'crow', 'deer', 'dog', 'dolphin', 'donkey', 'dragonfly', 'duck', 'eagle', 'elephant', 'flamingo', 'fly', 'fox', 'goat', 'goldfish', 'goose', 'gorilla', 'grasshopper', 'hamster', 'hare', 'hedgehog', 'hippopotamus', 'hornbill', 'horse', 'hummingbird', 'hyena', 'jellyfish', 'kangaroo', 'koala', 'ladybugs', 'leopard', 'lion', 'lizard', 'lobster', 'mosquito', 'moth', 'mouse', 'octopus', 'okapi', 'orangutan', 'otter', 'owl', 'ox', 'oyster', 'panda', 'parrot', 'pelecaniformes', 'penguin', 'pig', 'pigeon', 'porcupine', 'possum', 'raccoon', 'rat', 'reindeer', 'rhinoceros', 'sandpiper', 'seahorse', 'seal', 'shark', 'sheep', 'snake', 'sparrow', 'squid', 'squirrel', 'starfish', 'swan', 'tiger', 'turkey', 'turtle', 'whale', 'wolf', 'wombat', 'woodpecker', 'zebra']
number of classes: 90


In [8]:
classification = {f"Animal - Fold":[], 
                  f"Accuracy (animal, rest)": [], 
                  f"Precision (animal, rest)": [],
                    f"Recall (animal, rest)": [],
                    f"F-1 Score (animal, rest)": []}
for animal in animals:
    image_list, label_list = [] , []
    save_path = f'C:\\Users\\rudra\\OneDrive\\Desktop\\animals\\OvR3\\{animal}'
    train_path = f"C:\\Users\\rudra\\OneDrive\\Desktop\\animals\\OvR3\\{animal}"
    try:
        print("[INFO] Loading images ...")
        image_folder_list = listdir(train_path)
        for image_folder in image_folder_list:
            print(f"[INFO] Processing {image_folder} ...")
            images_list = listdir(f"{train_path}/{image_folder}/")
            for samples in images_list[:]:
                image_directory = f"{train_path}/{image_folder}/{samples}"
                if image_directory.endswith(".jpg")==True or image_directory.endswith(".JPG")==True or image_directory.endswith(".jpeg")==True:
                    # img = load_img(image_directory,target_size=(128,128))
                    # x = img_to_array(img,data_format = channels_last)  
                    image_list.append(img_to_array(load_img(image_directory,target_size=(128,128))))
                    label_list.append(image_folder)
    
        print("[INFO] image extract completed")  
        print("total train samples: ")
        print(len(image_list))
    except Exception as e:
        print(f"Error : {e}")
    X = np.array(image_list) / 225
    y = np.array(label_list)
    image_list, label_list = [] , []
    Y = labelencoder.fit_transform(y)
    labels = [
        f'{animal}',
        'rest' 
        ]
    cv = KFold(n_splits = 3, shuffle=True, random_state=42)
    fold_no = 1
    acc_per_fold = []
    loss_per_fold = []
    EPOCHS =20
    BATCH_SIZE = 32
    gc.collect()
    for train,test in cv.split(X,Y):
        model = create_model()
        print(f"[INFO] Training network for {animal} fold {fold_no}...")
        history = model.fit(X[train],Y[train], batch_size=BATCH_SIZE,
                                      steps_per_epoch=len(X[train]) // BATCH_SIZE,
                                      epochs=EPOCHS, 
                                      verbose=1)
        gc.collect()
        print(f"[INFO] Saving Model for {animal} fold {fold_no} ...")
        model.save(f"{save_path}/model_{animal}-fold"+str(fold_no)+".h5")
        scores = model.evaluate(X[test],Y[test],verbose=0)
        acc_per_fold.append(scores[1]*100)
        loss_per_fold.append(scores[0])
        y_probs = model.predict(X[test])
        # Get predicted labels
        yhat_probs = (y_probs > 0.5) 
        y_trues = Y[test]
        # Accuracy 
        accuracy = accuracy_score(y_trues, yhat_probs) 
        # Precision 
        precision = precision_score(y_trues, yhat_probs) 
        # Recall 
        recall = recall_score(y_trues, yhat_probs) 
        # F1-Score 
        f1 = f1_score(y_trues, yhat_probs)
        classification["Animal - Fold"].append(f'{animal} - {fold_no}')
        classification["Accuracy (animal, rest)"].append(accuracy)
        classification["Precision (animal, rest)"].append(precision)
        classification["Recall (animal, rest)"].append(recall)
        classification["F-1 Score (animal, rest)"].append(f1)
        gc.collect()
        fold_no = fold_no+1
df = pd.DataFrame(classification)
df.to_csv("Classification_report_of_all_OvR_classes.csv")

[INFO] Loading images ...
[INFO] Processing antelope ...
[INFO] Processing rest ...
[INFO] image extract completed
total train samples: 
3434
[INFO] Training network for antelope fold 1...
Epoch 1/20
71/71 [==============================] - 12s 48ms/step - loss: 0.4788 - accuracy: 0.7940
Epoch 2/20
71/71 [==============================] - 4s 48ms/step - loss: 0.3833 - accuracy: 0.8290
Epoch 3/20
71/71 [==============================] - 3s 48ms/step - loss: 0.2968 - accuracy: 0.8733
Epoch 4/20
71/71 [==============================] - 3s 48ms/step - loss: 0.2073 - accuracy: 0.9189
Epoch 5/20
71/71 [==============================] - 3s 48ms/step - loss: 0.1480 - accuracy: 0.9477
Epoch 6/20
71/71 [==============================] - 3s 48ms/step - loss: 0.1326 - accuracy: 0.9508
Epoch 7/20
71/71 [==============================] - 3s 48ms/step - loss: 0.0981 - accuracy: 0.9672
Epoch 8/20
71/71 [==============================] - 3s 48ms/step - loss: 0.1060 - accuracy: 0.9606
Epoch 9/20
71/71 [

C:\Users\rudra\AppData\Local\Temp\ipykernel_6820\3249705492.py:34: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


1/1 [==============================] - 0s 124ms/step


C:\Users\rudra\AppData\Local\Temp\ipykernel_6820\1767733992.py:15: RuntimeWarning: invalid value encountered in divide
  channel_image /= channel_image.std()
C:\Users\rudra\AppData\Local\Temp\ipykernel_6820\1767733992.py:18: RuntimeWarning: invalid value encountered in cast
  channel_image = np.clip(channel_image, 0, 255).astype('uint8')


[INFO] Training network for antelope fold 2...
Epoch 1/20
71/71 [==============================] - 5s 49ms/step - loss: 0.4315 - accuracy: 0.8046
Epoch 2/20
71/71 [==============================] - 3s 48ms/step - loss: 0.3175 - accuracy: 0.8675
Epoch 3/20
71/71 [==============================] - 3s 47ms/step - loss: 0.2156 - accuracy: 0.9101
Epoch 4/20
71/71 [==============================] - 3s 47ms/step - loss: 0.1656 - accuracy: 0.9362
Epoch 5/20
71/71 [==============================] - 3s 47ms/step - loss: 0.1390 - accuracy: 0.9446
Epoch 6/20
71/71 [==============================] - 3s 47ms/step - loss: 0.1677 - accuracy: 0.9322
Epoch 7/20
71/71 [==============================] - 3s 47ms/step - loss: 0.1302 - accuracy: 0.9521
Epoch 8/20
71/71 [==============================] - 3s 47ms/step - loss: 0.0719 - accuracy: 0.9756
Epoch 9/20
71/71 [==============================] - 3s 47ms/step - loss: 0.0524 - accuracy: 0.9840
Epoch 10/20
71/71 [==============================] - 3s 47ms/s

...

1/1 [==============================] - 0s 78ms/step


C:\Users\rudra\AppData\Local\Temp\ipykernel_6820\1767733992.py:15: RuntimeWarning: invalid value encountered in divide
  channel_image /= channel_image.std()
C:\Users\rudra\AppData\Local\Temp\ipykernel_6820\1767733992.py:18: RuntimeWarning: invalid value encountered in cast
  channel_image = np.clip(channel_image, 0, 255).astype('uint8')


[INFO] Training network for zebra fold 3...
Epoch 1/20
71/71 [==============================] - 4s 48ms/step - loss: 0.3128 - accuracy: 0.8666
Epoch 2/20
71/71 [==============================] - 3s 48ms/step - loss: 0.1591 - accuracy: 0.9398
Epoch 3/20
71/71 [==============================] - 3s 47ms/step - loss: 0.1097 - accuracy: 0.9642
Epoch 4/20
71/71 [==============================] - 3s 48ms/step - loss: 0.1067 - accuracy: 0.9646
Epoch 5/20
71/71 [==============================] - 3s 47ms/step - loss: 0.0633 - accuracy: 0.9792
Epoch 6/20
71/71 [==============================] - 3s 47ms/step - loss: 0.0483 - accuracy: 0.9819
Epoch 7/20
71/71 [==============================] - 3s 47ms/step - loss: 0.0307 - accuracy: 0.9912
Epoch 8/20
71/71 [==============================] - 3s 48ms/step - loss: 0.0284 - accuracy: 0.9920
Epoch 9/20
71/71 [==============================] - 3s 48ms/step - loss: 0.0206 - accuracy: 0.9956
Epoch 10/20
71/71 [==============================] - 3s 48ms/step

C:\Users\rudra\AppData\Local\Temp\ipykernel_6820\3249705492.py:34: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


1/1 [==============================] - 0s 81ms/step
